In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import matthews_corrcoef, make_scorer

In [2]:
# reading in sample data and then reducing the no of columns on the avaialbity of data
numerical_data = pd.read_csv("../bosch/train_numeric_sample.csv")
# considering columns with only > 5000 entries
num_data_des = numerical_data.describe()
column_fill = num_data_des.ix['count']
column_fill = column_fill.drop('Id')
column_fill = column_fill.drop('Response')
columns = column_fill[column_fill>5000]
# finding the columns is done on only sample data. Now read the full train data snd train the classifier.


/home/manpun/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [3]:
#Sample data for processing
num_data_concise = numerical_data[columns.index]
y_data_concise = numerical_data['Response']

In [ ]:
num_data_concise = pd.read_csv("../bosch/train_numeric.csv", usecols = columns.index)
y_data_concise = pd.read_csv('../bosch/train_numeric.csv', usecols = ['Response'])

In [4]:
# filling the NAn with means for now.
# think of any pattern to fit the missing data
num_data_concise = num_data_concise.fillna(num_data_concise.mean())
X = num_data_concise;
#Y = y_data_concise.astype(int)
Y , _ = pd.factorize(y_data_concise)
# we got the final clean data to send it to the classifier.

In [20]:
def threshold_finder(clf, Xvalid, Yvalid):
    class_labels = clf.classes_
    print (class_labels)
    Ypred = clf.predict_proba(Xvalid)
    Yv = Yvalid.astype(int)
    best_score = 0
    best = -2
    for t in np.arange(0.4,0.8,0.1):
        print (t)
        Ypre = np.array(Ypred>t)
        Ypr = Ypre[:,1]
        Yp = Ypr.astype(int)
        print (Yp.mean())
        score = matthews_corrcoef(Yv, Yp)
        print (score)
        if (score > best_score):
            best_score = score
            best = t
    print ('Best score :', best_score , 'threshold : ', best)
    return best_score

def tFinder(Yvalid,Yprob):
    Yv = Yvalid.astype(int)
    best_score = -2
    best = 0
    for t in np.arange(0.4,0.8,0.1):
        print (t)
        Ypre = np.array(Yprob>t)
        Yp = Ypre.astype(int)
        print (Yp.mean())
        score = matthews_corrcoef(Yv, Yp)
        print (score)
        if (score > best_score):
            best_score = score
            best = t
    return best_score


tmcc = make_scorer(tFinder)
#scoring mechanism to be passed to Gridsearchcv
mcc = make_scorer(matthews_corrcoef)

In [25]:
# defining a function to cross validate the training;
def cv_optimize(clf, parameters, Xtrain, Ytrain, nfolds=4):
    gs = GridSearchCV(clf, param_grid=parameters, cv = nfolds, n_jobs = 1, pre_dispatch= '2*n_jobs', scoring = 'average_precision')
    gs.fit(Xtrain,Ytrain)
    print ("Best params")
    print (gs.best_params_)
    best = gs.best_estimator_
    return best

def classify(clf, parameters,X,Y, train_size=0.8):
    # first splitting the availabe data set to train and test 
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,train_size=train_size) 
    clf = cv_optimize(clf, parameters, Xtrain, Ytrain)
    clf.set_params(n_jobs=4)
    clf=clf.fit(Xtrain, Ytrain)
    test_score = threshold_finder(clf, Xtest, Ytest)
    print ('Final test score' , test_score)
    return clf

In [26]:
rf = classify(RandomForestClassifier(n_jobs=4), {'n_estimators' : [100, 200, 400, 800], 'max_depth': [3,6,9,12]},X , Y, train_size =0.8)

Best params
{'n_estimators': 200, 'max_depth': 6}
[0 1]
0.4
0.0
0.0
0.5
0.0
0.0
0.6
0.0
0.0
0.7
0.0
0.0
Best score : 0 threshold :  -2
Final test score 0


In [ ]:
rf = RandomForestClassifier(n_estimators = 1500, max_depth = 12, n_jobs =4, class_weight ='balanced')
rf.fit(X,Y)

In [ ]:
Ypred = rf.predict_proba(X)

In [ ]:
threshold_finder(rf,X,Y)

In [ ]:
Ypred = Ypred>0.5
type(Ypred)
Ypr = Ypred[:,1]
Ypr.astype(int)

In [ ]:
Y.astype(int)
matthews_corrcoef(Y,Ypr)

In [ ]:
type_true = type_of_target(y_true)
type_pred = type_of_target(y_pred)

In [ ]:
X = pd.DataFrame()
num_data_concise = pd.DataFrame()

Reading the test data to predict the failures. First we have to read in all the data, then take the only sleceted columns as per trainig. Then fill Na's with mean adn then classify/predict.

In [ ]:
test_data = pd.read_csv('../bosch/test_numeric.csv', usecols = columns.index)
test_data = test_data.fillna(test_data.mean())
results = rf.predict(test_data)

In [ ]:
resultsp = rf.predict_proba(test_data)
t = [x[1] for x in resultsp]
t =np.array(t)
t = t > 0.65
t = t.astype(int)
t.mean()

In [ ]:
test_id = pd.read_csv('../bosch/test_numeric.csv', usecols = ['Id'])
submission = pd.DataFrame()
submission['Id'] = test_id['Id']
submission['Response'] = t
submission.to_csv('balanced_rf_100_3_65.csv', index = False)

In [ ]:
%reset_selective test_data